In [63]:
from IPython.display import display
import math
'''import piplite
await piplite.install("bqplot")
await piplite.install("ipywidgets")'''
import bqplot
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
from scipy.integrate import solve_ivp

In [64]:
#Variables

n = 2 #Hill coefficient
Km = 40 #monomeres per cell
beta = 0.2
alpha0 = 0.2164 
alpha = 216.404
tau_mRNA = 2
tau_prot = 10
kd_m = math.log(2)/tau_mRNA
kd_p = math.log(2)/tau_prot
k_tl = 6.93

CmtetR0 = 1
Cmlambda0 = 4
CmlacI0 = 1
CptetR0 = 2
Cplambda0 = 1
CplacI0 = 1

In [65]:
#Calculating data for plots
    
def repressilator_model(t, data, alpha0, alpha, kd_m, kd_p, k_tl):
    m = data[:3]
    p = data[3:]
    dm0 = - kd_m*m[0] + alpha*(Km**n)/((Km**n)+(p[2])**n) + alpha0
    dm1 = - kd_m*m[1] + alpha*(Km**n)/((Km**n)+(p[0])**n) + alpha0
    dm2 = - kd_m*m[2] + alpha*(Km**n)/((Km**n)+(p[1])**n) + alpha0
    dp0 = - kd_p*p[0] + k_tl*m[0]
    dp1 = - kd_p*p[1] + k_tl*m[1]
    dp2 = - kd_p*p[2] + k_tl*m[2]
    return [dm0,dm1,dm2,dp0,dp1,dp2]
    
t0 = 0.0
tf = 1000.0
time = np.linspace(0.0,1000.0,10000)
initval = np.array([CmtetR0,Cmlambda0,CmlacI0,CptetR0,Cplambda0,CplacI0])
res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(alpha0,alpha,kd_m,kd_p,k_tl))
res = dict(res)
time = res['t']
y = res['y']

ylacI = y[5]
ytetR = y[3]
ylambda = y[4]

fig = plt.figure(title="Repressilator model", legend_location="top-left")

line_chart = plt.plot(x=time, y=[ylacI, ytetR, ylambda],
                     labels=["LacI","TetR", "cI"], colors=['blue', 'green', 'red'],
                     display_legend=True)

plt.xlabel("Time, min")
plt.ylabel("Number of proteins")

In [66]:
#Callback
def update_plot(tM, tP, k_tl, a, a0, CmlacI0, CmtetR0, CmcI0, CplacI0, CptetR0, CpcI0):
    kd_m = math.log(2)/tM
    kd_p = math.log(2)/tP
    t0 = 0.0
    tf = 1000.0
    time = np.linspace(0.0,1000.0,10000)
    initval = np.array([CmtetR0,CmcI0,CmlacI0,CptetR0,CpcI0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(a0,a,kd_m,kd_p,k_tl))
    res = dict(res)
    time = res['t']
    y = res['y']
    ylacI = y[5]
    ytetR = y[3]
    ylambda = y[4]
    line_chart.y = [ylacI, ytetR, ylambda]

In [67]:
#widgets
wdj = widgets.interactive(update_plot, tM = widgets.FloatText(value=tau_mRNA, description="tau_mRNA:", disabled=False),
tP = widgets.FloatText(value=tau_prot, description="tau_prot:", disabled=False),
k_tl = widgets.FloatText(value=k_tl, description="k_tl:", disabled=False),                          
a = widgets.FloatText(value=alpha, description="alpha:", disabled=False),
a0 = widgets.FloatText(value=alpha0, description="alpha0:", disabled=False),
CmlacI0 = widgets.FloatText(value=CmlacI0, description="CmlacI0:", disabled=False),
CmtetR0 = widgets.FloatText(value=CmtetR0, description="CmtetR0:", disabled=False),
CmcI0 = widgets.FloatText(value=Cmlambda0, description="Cmlambda0:", disabled=False),
CplacI0 = widgets.FloatText(value=CplacI0, description="CplacI0:", disabled=False),
CptetR0 = widgets.FloatText(value=CptetR0, description="CptetR0:", disabled=False),
CpcI0 = widgets.FloatText(value=Cplambda0, description="Cplambda0:", disabled=False))

In [70]:
display(widgets.HBox([fig, wdj]))